In [1]:
import pandas as pd
from pathlib import Path

filename = Path("cleaned_results/tuned_aggregated_results")
original = pd.read_csv(f'{filename}_clean.csv')
missing = pd.read_csv(f'{filename}_missing.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_results/tuned_aggregated_results_with_default_original.csv'

In [16]:
merged = pd.merge(original, missing, on=['alg_name', 'dataset_name'], how='right', suffixes=('_original', '_missing'))
cols = ['alg_name', 'dataset_name']
cols.extend([x for x in merged.columns if ("accuracy" in x.lower()) and ("mean" in x.lower())])
merged = merged[cols]
# Group by dataset_name and alg_name, then rank within each group
merged['updated_rank_original'] = merged.groupby(
    ['dataset_name']
)['Accuracy__test_mean_original'].rank(ascending=False, method='dense')

merged.sort_values(by=['dataset_name', 'updated_rank_original'], inplace=True)
merged

,alg_name,dataset_name,Accuracy__test_mean_original,Accuracy__test_mean_min_original,Accuracy__test_mean_max_original,normalized_Accuracy__test_mean_original,Accuracy_rank_mean_original,Accuracy__test_mean_missing,train_per_1000_inst_mean_Accuracy,Accuracy__test_mean_min_missing,Accuracy__test_mean_max_missing,normalized_Accuracy__test_mean_missing,Accuracy_rank_mean_missing,updated_rank_original
88,XGBoost,openml__Bioresponse__9910,0.796848,0.727272,0.796848,1.000000,1.0,0.793916,2.932279,0.759794,0.794717,0.977072,2.0,1.0
0,CatBoost,openml__Bioresponse__9910,0.795521,0.727272,0.796848,0.980938,2.0,0.794717,24.085377,0.759794,0.794717,1.000000,1.0,2.0
120,XGBoost (default),openml__Bioresponse__9910,0.792853,0.727272,0.796848,0.942590,4.0,0.793653,4.491103,0.759794,0.794717,0.969538,3.0,3.0
32,CatBoost (default),openml__Bioresponse__9910,0.782994,0.727272,0.796848,0.800881,8.0,0.786985,4.940433,0.759794,0.794717,0.778599,4.0,4.0
64,ResNet,openml__Bioresponse__9910,0.776065,0.727272,0.796848,0.701301,11.0,0.770726,4.538032,0.759794,0.794717,0.313012,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,CatBoost (default),openml__splice__45,0.949530,0.550470,0.967398,0.957143,15.0,0.960502,22.755163,0.959248,0.960815,0.800000,2.0,4.0
31,CatBoost,openml__vehicle__53,0.768347,0.252941,0.842745,0.873860,7.0,0.758866,30.289744,0.748193,0.764748,0.644670,3.0,1.0
63,CatBoost (default),openml__vehicle__53,0.747129,0.252941,0.842745,0.837885,11.0,0.764748,32.339726,0.748193,0.764748,1.000000,1.0,2.0
119,XGBoost,openml__vehicle__53,0.744678,0.252941,0.842745,0.833729,12.0,0.760028,8.712856,0.748193,0.764748,0.714890,2.0,3.0


In [17]:
merged.to_csv(f'{filename}_merged.csv', index=False)
column_mappings =  {
    'updated_rank_original': 'Recomputed McElfresh Rank',
    'Accuracy__test_mean_original': 'McElfresh Mean Test Accuracy',
    'Accuracy__test_mean_missing': 'Missing Mean Test Accuracy',
    'Accuracy_rank_mean_missing': 'Missing Rank',
}
merged.rename(columns=column_mappings, inplace=True)
merged[column_mappings.values()]